# 회귀 분석


## 전복 고리수 추정 신경망

### 전복 데이터
아발로니 데이터은 전복에 대한 8가지 특정값과 현미경을 통해 측정한 고리 수를 담고 있다.  
- 성별
- 키
- 지름
- 높이
- 전체 무게
- 몸통 무게
- 내장 무게
- 껍질 무게
- 껍질의 고리 수

In [21]:
!wget -O abalone.csv https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data

--2021-01-28 08:07:48--  https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191873 (187K) [application/x-httpd-php]
Saving to: ‘abalone.csv’

abalone.csv         100%[===================>] 187.38K   907KB/s    in 0.2s    

2021-01-28 08:07:49 (907 KB/s) - ‘abalone.csv’ saved [191873/191873]



### 파이썬 모듈 불러들이기

In [22]:
import numpy as np
import csv
import time

np.random.seed(1234)

def randomize():
    np.random.seed(time.time())

### 하이퍼파라미터값의 정의

In [23]:
# 정규분포 난수값의 평균과 표준편차 정의
RND_MEAN = 0
RND_STD = 0.0030

# 학습률 정의
LEARNING_RATE = 0.001

### 실험용 메인 함수 정의

In [24]:
def abalone_exec(epoch_count=10, mb_size=10, report=1): # 학습 횟수, 미니배치 크기, 중간 보고 주기
    load_abalone_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

### 데이터 적재 함수 정의

In [25]:
def load_abalone_dataset():
    with open('abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10, 1
    data = np.zeros([len(rows), input_cnt+output_cnt])

    for n, row in enumerate(rows):
        if row[0] == 'I': data[n, 0] = 1
        if row[0] == 'M': data[n, 1] = 1
        if row[0] == 'F': data[n, 2] = 1
        data[n, 3:] = row[1:]

### 파라미터 초기화 함수 정의

In [26]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD,[input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

### 학습 및 평가 함수 정의

In [27]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

### 학습 및 평가  데이터 획득 함수 정의

In [28]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

### 학습 실행 함수와 평가 실행 함수 정의

In [29]:
def run_train(x, y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)
    
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

### 단층 퍼셉트론에 대한 순전파 및 역전파 함수 정의

In [30]:
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x

def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

### 후처리 과정에 대한 순전파 및 역전파 함수 정의

In [31]:
def forward_postproc(output, y):
    diff = output - y
    square = np.square(diff)
    loss = np.mean(square)
    return loss, diff

def backprop_postproc(G_loss, diff):
    shape = diff.shape
    
    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1

    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff
    
    return G_output

In [32]:
def backprop_postproc_oneline(G_loss, diff): 
    return 2 * diff / np.prod(diff.shape)

### 정확도 계산 함수 정의

In [33]:
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output - y)/y))
    return 1 - mdiff

## 실행하기

### 전복 고리 수 측정 신경망 실행

In [37]:
abalone_exec()

Epoch 1: loss=33.875, accuracy=0.557/0.812
Epoch 2: loss=8.226, accuracy=0.820/0.814
Epoch 3: loss=7.582, accuracy=0.812/0.809
Epoch 4: loss=7.475, accuracy=0.808/0.811
Epoch 5: loss=7.395, accuracy=0.810/0.809
Epoch 6: loss=7.328, accuracy=0.808/0.810
Epoch 7: loss=7.269, accuracy=0.808/0.811
Epoch 8: loss=7.217, accuracy=0.808/0.812
Epoch 9: loss=7.175, accuracy=0.810/0.810
Epoch 10: loss=7.135, accuracy=0.809/0.810

Final Test: final accuracy = 0.810


### 학습된 파라미터값 확인

In [38]:
print(weight)
print(bias)

[[1.02697603]
 [1.47450981]
 [1.66960135]
 [2.04468668]
 [1.62513525]
 [0.60292627]
 [2.39993815]
 [0.54107313]
 [0.46878034]
 [1.01969382]]
[4.16894769]


### 하이퍼파라미터 바꾸어 실행하기

In [39]:
LEARNING_RATE = 0.1
abalone_exec(epoch_count=100, mb_size=100, report=20)

Epoch 20: loss=5.804, accuracy=0.825/0.831
Epoch 40: loss=5.259, accuracy=0.834/0.828
Epoch 60: loss=5.056, accuracy=0.837/0.838
Epoch 80: loss=4.950, accuracy=0.838/0.840
Epoch 100: loss=4.910, accuracy=0.840/0.826

Final Test: final accuracy = 0.826
